In [1]:
import numpy as np
import pandas as pd 
import geopy as gp
from geopy import distance
from geopy.geocoders import Nominatim 

In [2]:
df = pd.read_csv('SubwayStations.csv')
df = df.fillna(0)
cols = ['Line 1', 'Line 2', 'Line 3', 'Line 4']
df[cols] = df[cols].applymap(np.int64)
df = df.sort_values(by=['Station'])
df = df.reset_index(drop=True)
df

,Station,Line 1,Line 2,Line 3,Line 4
0,Bathurst Station,0,1,0,0
1,Bay Station,0,1,0,0
2,Bayview Station,0,0,0,1
3,Bessarion Station,0,0,0,1
4,Bloor Station,1,1,0,0
...,...,...,...,...,...
70,Wilson Station,1,0,0,0
71,Woodbine Station,0,1,0,0
72,York Mills Station,1,0,0,0
73,York University Station,1,0,0,0


In [3]:
stops = pd.read_csv('stops.csv')
station_name = df['Station']
subway_stops = stops.iloc[0:1,:]
for i in range(len(station_name)):
    ss = stops[stops['stop_name'].str.startswith(station_name[i])]
    subway_stops = pd.concat([subway_stops, ss], ignore_index=True)
subway_stops = subway_stops.iloc[1:,:]
subway_stops['stop_name'] = subway_stops['stop_name'].str.replace( ' - Southbound Platform', '')
subway_stops['stop_name'] = subway_stops['stop_name'].str.replace( ' - Northbound Platform', '')
subway_stops['stop_name'] = subway_stops['stop_name'].str.replace( ' - Westbound Platform', '')
subway_stops['stop_name'] = subway_stops['stop_name'].str.replace( ' - Eastbound Platform', '')
# subway_stops = subway_stops[~subway_stops['stop_name'].str.contains('Platform')]

In [4]:
ss_name = subway_stops['stop_name'].unique()
new_list = list(set(ss_name).difference(station_name))
new_list2 = list(set(station_name).difference(ss_name))
new_list, new_list2

(['Union Station - Nb Platform Towards Vaughan Metropolitan Centre',
  'Union Station Towards Finch',
  'Kipling Station - Subway Platform',
  'McCowan Station - Platform',
  'Sheppard-Yonge Station - Subway Platform',
  'Kennedy Station - Subway Platform',
  'Finch Station - Subway Platform',
  'Don Mills Station - Subway Platform',
  'Kennedy Station - Platform',
  'Sheppard West Station - Subway Platform'],
 [])

In [5]:
subway_stops['stop_name'] = subway_stops['stop_name'].str.replace( ' - Subway Platform', '')
subway_stops['stop_name'] = subway_stops['stop_name'].str.replace( ' Towards Finch', '')
subway_stops['stop_name'] = subway_stops['stop_name'].str.replace( ' - Platform', '')
subway_stops['stop_name'] = subway_stops['stop_name'].str.replace( ' - Nb Platform Towards Vaughan Metropolitan Centre', '')
ss_name = subway_stops['stop_name'].unique()

In [6]:
ss_stops = subway_stops.iloc[0:1,:]
for i in range(len(ss_name)):
    ss = subway_stops[subway_stops['stop_name'].str.startswith(ss_name[i])]
    ss_stops = pd.concat([ss_stops, ss.iloc[0:1,:]], ignore_index=True)
ss_stops = ss_stops.iloc[1:,:]
ss_stops = ss_stops.sort_values(by=['stop_name'])

In [7]:
ss_stops = ss_stops.reset_index(drop=True)
ss_stops 

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,14141,14617,Bathurst Station,NaN,43.666061,-79.411065,NaN,NaN,NaN,NaN,NaN,1
1,14484,13757,Bay Station,NaN,43.670149,-79.390089,NaN,NaN,NaN,NaN,NaN,2
2,14531,13844,Bayview Station,NaN,43.766872,-79.386703,NaN,NaN,NaN,NaN,NaN,1
3,14532,13845,Bessarion Station,NaN,43.769202,-79.376312,NaN,NaN,NaN,NaN,NaN,1
4,14414,13864,Bloor Station,NaN,43.670049,-79.385389,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
70,14316,14799,Wilson Station,NaN,43.734475,-79.450920,NaN,NaN,NaN,NaN,NaN,2
71,14321,14804,Woodbine Station,NaN,43.686440,-79.312471,NaN,NaN,NaN,NaN,NaN,1
72,14407,13792,York Mills Station,NaN,43.743248,-79.405991,NaN,NaN,NaN,NaN,NaN,1
73,15694,15666,York University Station,NaN,43.774097,-79.499888,NaN,NaN,NaN,NaN,NaN,1


In [8]:
new_list = list(set(ss_name).difference(station_name))
new_list2 = list(set(station_name).difference(ss_name))
new_list, new_list2

([], [])

In [9]:
df['lat'] = ss_stops['stop_lat']
df['lon'] = ss_stops['stop_lon']
df

,Station,Line 1,Line 2,Line 3,Line 4,lat,lon
0,Bathurst Station,0,1,0,0,43.666061,-79.411065
1,Bay Station,0,1,0,0,43.670149,-79.390089
2,Bayview Station,0,0,0,1,43.766872,-79.386703
3,Bessarion Station,0,0,0,1,43.769202,-79.376312
4,Bloor Station,1,1,0,0,43.670049,-79.385389
...,...,...,...,...,...,...,...
70,Wilson Station,1,0,0,0,43.734475,-79.450920
71,Woodbine Station,0,1,0,0,43.686440,-79.312471
72,York Mills Station,1,0,0,0,43.743248,-79.405991
73,York University Station,1,0,0,0,43.774097,-79.499888


In [10]:
print(df['lat'].isnull().sum())
print(df['lon'].isnull().sum())

0
0


In [11]:
ss_name

array(['Bathurst Station', 'Bay Station', 'Bayview Station',
       'Bessarion Station', 'Bloor Station', 'Broadview Station',
       'Castle Frank Station', 'Chester Station', 'Christie Station',
       'College Station', 'Coxwell Station', 'Davisville Station',
       'Don Mills Station', 'Donlands Station', 'Downsview Park Station',
       'Dufferin Station', 'Dundas Station', 'Dundas West Station',
       'Dupont Station', 'Eglinton Station', 'Eglinton West Station',
       'Ellesmere Station', 'Finch Station', 'Finch West Station',
       'Glencairn Station', 'Greenwood Station', 'High Park Station',
       'Highway 407 Station', 'Islington Station', 'Jane Station',
       'Keele Station', 'Kennedy Station', 'King Station',
       'Kipling Station', 'Lansdowne Station', 'Lawrence East Station',
       'Lawrence Station', 'Lawrence West Station', 'Leslie Station',
       'Main Street Station', 'McCowan Station', 'Midland Station',
       'Museum Station', 'North York Centre Station

# Calculate Distance 

## 1. Restaurants

In [12]:
# find subway station within 5 km 
rst_df = pd.read_excel('toronto_restaurants.xlsx')
rst_df.head()
rst_df = rst_df[rst_df['Restaurant Latitude'].notna()]
rst_df = rst_df[rst_df['Restaurant Longitude'].notna()]
for i in range (len(df)):
    ss_dist = []
    coords_ss = (df.iloc[i,5], df.iloc[i,6])
    for j in range (len(rst_df)): 
        coords_r = (rst_df.iloc[j,7], rst_df.iloc[j,8]) 
        if distance.distance(coords_ss, coords_r).km <=5:
            ss_dist.append(1)
            
        else: 
            ss_dist.append(0)
    rst_df[df.iloc[i,0]] = ss_dist

rst_df    

,Category,Restaurant Address,Restaurant Name,Restaurant Phone,Restaurant Price Range,Restaurant Website,Restaurant Yelp URL,Restaurant Latitude,Restaurant Longitude,Bathurst Station,...,Union Station,Vaughan Metropolitan Centre Station,Victoria Park Station,Warden Station,Wellesley Station,Wilson Station,Woodbine Station,York Mills Station,York University Station,Yorkdale Station
0,Afghan,"14 Prince Arthur Avenue\nToronto, ON M5R 1A9",The Host,(416) 962-4678,$11-30,welcometohost.com,https://www.yelp.ca/adredir?ad_business_id=OFA...,43.669935,-79.395858,1,...,1,0,0,0,1,0,0,0,0,0
1,Afghan,"259 Wellington St W\nToronto, ON M5V",Aanch Modernist Indian Cuisine,(647) 558-1508,$11-30,aanch.ca,https://www.yelp.ca/adredir?ad_business_id=SZu...,43.644708,-79.390670,1,...,1,0,0,0,1,0,0,0,0,0
2,Afghan,"736 Bay Street\nToronto, ON M5G 2J8",Silk Road Kabob House,NaN,Under $10,NaN,https://www.yelp.ca/biz/silk-road-kabob-house-...,43.659816,-79.385591,1,...,1,0,0,0,1,0,0,0,0,0
3,Afghan,"691 Yonge Street\nToronto, ON M4Y 2B2",Naan & Kabob,(416) 972-6623,$11-30,naanandkabob.ca,https://www.yelp.ca/biz/naan-and-kabob-toronto-5,43.669058,-79.386100,1,...,1,0,0,0,1,0,0,0,0,0
4,Afghan,"66 Overlea Boulevard\nUnit 62\nToronto, ON M4H...",Afghan Cuisine,(416) 422-5858,$11-30,afghancuisinerestaurant.com,https://www.yelp.ca/biz/afghan-cuisine-toronto,43.708070,-79.341508,0,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15816,Waffles,"100 Steeles Avenue W\nUnit 5\nVaughan, ON L4J 7Y1",Red Bean Waffle House,(905) 597-5589,Under $10,NaN,https://www.yelp.ca/biz/red-bean-waffle-house-...,43.797877,-79.425598,0,...,0,0,0,0,0,0,0,0,0,0
15817,Waffles,"106 Mutual Street\nUnit 8\nToronto, ON M5B 2R7",Page One,(416) 663-4920,$11-30,pageonecafe.com,https://www.yelp.ca/adredir?ad_business_id=HkP...,43.657853,-79.376254,1,...,1,0,0,0,1,0,0,0,0,0
15818,Waffles,"507 Mount Pleasant Rd\nToronto, ON M4S 2L9",Bread & Butter,(647) 691-4910,Under $10,breadbutter.ca,https://www.yelp.ca/adredir?ad_business_id=U0m...,43.701638,-79.387440,1,...,0,0,0,0,1,0,0,1,0,0
15819,Waffles,"2674 Yonge Street\nToronto, ON M4N 2H7",Sugar Miracles,(647) 350-2524,Under $10,sugarmiracles.com,https://www.yelp.ca/adredir?ad_business_id=Hn-...,43.716805,-79.400696,0,...,0,0,0,0,0,1,0,1,0,1


## 2. Point of Interests
Since this dataset does not include the latitude and longitude of the address, we will find that using geopy.

In [13]:
poi_df = pd.read_excel('Places of Interest and Attractions.xlsx')
poi_df = poi_df[poi_df['ADDRESS_FULL'].notna()]
poi_df['ADDRESS_FULL'] = poi_df['ADDRESS_FULL'].astype(str) + ' Toronto Canada'
poi_df['ADDRESS_FULL']

0       170 Princes' Blvd Toronto Canada
1           77 Wynford Dr Toronto Canada
2               40 Bay St Toronto Canada
3         750 Spadina Ave Toronto Canada
4           2901 Yonge St Toronto Canada
                     ...                
170         1 Dundas St E Toronto Canada
171     235 Queens Quay W Toronto Canada
172      3401 Dufferin St Toronto Canada
173    50 Tank House Lane Toronto Canada
174        165 Front St E Toronto Canada
Name: ADDRESS_FULL, Length: 175, dtype: object

In [14]:
geolocator = Nominatim(user_agent = "geoapiExercises")
#find lat and long
for i in range(len(poi_df)): 
    place = poi_df.iloc[i,13]
    location = geolocator.geocode(place)
    if location:
        lat=location.latitude
        lon=location.longitude
    else :
        lat = np.nan
        lon = np.nan
    poi_df.iloc[i,27] = lat
    poi_df.iloc[i,28] = lon 


In [15]:
print(poi_df['LATITUDE'])
print(poi_df['LONGITUDE'])

0      43.632833
1      43.725394
2      43.643430
3      43.666204
4      43.684011
         ...    
170    43.656263
171    43.638032
172    43.725624
173    43.650874
174    43.650050
Name: LATITUDE, Length: 175, dtype: float64
0     -79.422884
1     -79.332334
2     -79.379059
3     -79.404189
4     -79.392408
         ...    
170   -79.380763
171   -79.383156
172   -79.452308
173   -79.357662
174   -79.368863
Name: LONGITUDE, Length: 175, dtype: float64


In [16]:
# find subway station within 5 km 
poi_df = poi_df[poi_df['LATITUDE'].notna()]
poi_df = poi_df[poi_df['LONGITUDE'].notna()]
for i in range (len(df)):
    ss_dist = []
    coords_ss = (df.iloc[i,5], df.iloc[i,6])
    for j in range (len(poi_df)): 
        coords_poi = (poi_df.iloc[j,27], poi_df.iloc[j,28]) 
        if distance.distance(coords_ss, coords_poi).km <=5:
            ss_dist.append(1)
            
        else: 
            ss_dist.append(0)
    poi_df[df.iloc[i,0]] = ss_dist


poi_df.head()

,_id,ID,ADDRESS_INFO,NAME,CATEGORY,PHONE,EMAIL,WEBSITE,GEOID,RECEIVED_DATA_DATE,...,Union Station,Vaughan Metropolitan Centre Station,Victoria Park Station,Warden Station,Wellesley Station,Wilson Station,Woodbine Station,York Mills Station,York University Station,Yorkdale Station
0,1,16,NaN,BMO Field,Sports / Entertainment Venue,416-815-5982,NaN,www.bmofield.com,20229243.0,NaN,...,1,0,0,0,1,0,0,0,0,0
1,2,1,NaN,Aga Khan Museum,Museum,416-646-4677,NaN,www.agakhanmuseum.org,10142948.0,NaN,...,0,0,1,1,0,0,1,0,0,0
2,3,2,NaN,Air Canada Centre,Sports / Entertainment Venue,416-815-5500,NaN,www.theaircanadacentre.com/,7929257.0,NaN,...,1,0,0,0,1,0,0,0,0,0
3,4,3,NaN,Al Green Theatre (at the Miles Nadal Jewish Co...,Performing Arts,416-924-6211,NaN,www.algreentheatre.ca/,8418224.0,NaN,...,1,0,0,0,1,0,0,0,0,0
4,5,4,NaN,Alexander Muir Memorial Gardens,Nature/ Park,416-338-4386,NaN,http://www1.toronto.ca/parks/prd/facilities/co...,10154587.0,NaN,...,1,0,0,0,1,0,0,0,0,0


## 3. Making Space for Culture

In [17]:
cul_df = pd.read_excel('making-space-for-culture-tabular-data-excel.xlsx')
cul_df = cul_df[cul_df['Full Address'].notna()]
geolocator = Nominatim(user_agent = "geoapiExercises")
#find lat and long
lat_list = []
lon_list = []
for i in range(len(cul_df)): 
    place = cul_df.iloc[i,1]
    location = geolocator.geocode(place)
    if location:
        lat=location.latitude
        lon=location.longitude
    else :
        lat = np.nan
        lon = np.nan
    lat_list.append(lat)
    lon_list.append(lon)
    
cul_df['LATITUDE'] = lat_list
cul_df['LONGITUDE'] = lon_list

In [18]:
# find subway station within 5 km 
cul_df = cul_df[cul_df['LATITUDE'].notna()]
cul_df = cul_df[cul_df['LONGITUDE'].notna()]
for i in range (len(df)):
    ss_dist = []
    coords_ss = (df.iloc[i,5], df.iloc[i,6])
    for j in range (len(cul_df)): 
        coords_cul = (cul_df.iloc[j,16], cul_df.iloc[j,17]) 
        if distance.distance(coords_ss, coords_cul).km <=5:
            ss_dist.append(1)
            
        else: 
            ss_dist.append(0)
    cul_df[df.iloc[i,0]] = ss_dist


cul_df.head()

,FACILITY NAME,Full Address,Street #,Street Name,Suite,City,Province,Postal Code,Ward,Performance,...,Union Station,Vaughan Metropolitan Centre Station,Victoria Park Station,Warden Station,Wellesley Station,Wilson Station,Woodbine Station,York Mills Station,York University Station,Yorkdale Station
0,Thistletown CC,"925 Albion Road, Toronto, ON, M9V 1A6",925,Albion Road,NaN,Toronto,ON,M9V 1A6,1,1.0,...,0,0,0,0,0,0,0,0,0,0
4,Humber Arboretum Gardens,"203 Humber College Boulevard, Toronto, ON, M9W...",203,Humber College Boulevard,NaN,Toronto,ON,M9W 5L7,1,NaN,...,0,0,0,0,0,0,0,0,0,0
6,North Albion Collegiate Institute,"2580 Kipling Avenue, Toronto, ON, M9V 3B2",2580,Kipling Avenue,NaN,Toronto,ON,M9V 3B2,1,1.0,...,0,0,0,0,0,0,0,0,0,0
7,Smithfield Community Services,"175 Mount Olive Drive, Toronto, ON, M9V 2E3",175,Mount Olive Drive,NaN,Toronto,ON,M9V 2E3,1,NaN,...,0,0,0,0,0,0,0,0,0,0
13,Toronto Congress Centre,"650 Dixon Road, Toronto, ON, M9W 1J1",650,Dixon Road,NaN,Toronto,ON,M9W 1J1,2,1.0,...,0,0,0,0,0,0,0,0,0,0


## 4. Public Art Work Asset 2019

In [27]:
pa19_df = pd.read_csv('public-art-work-asset-2019_cleaned.csv')
pa19_df = pa19_df[pa19_df['ADDRESS'].notna()]
pa19_df['ADDRESS'] = pa19_df['ADDRESS'].astype(str) + ' Toronto Canada'
geolocator = Nominatim(user_agent = "geoapiExercises")
#find lat and long
lat_list = []
lon_list = []
for i in range(len(pa19_df)): 
    place = pa19_df.iloc[i,4]
    location = geolocator.geocode(place)
    if location:
        lat=location.latitude
        lon=location.longitude
    else :
        lat = np.nan
        lon = np.nan
    lat_list.append(lat)
    lon_list.append(lon)
    
pa19_df['LATITUDE'] = lat_list
pa19_df['LONGITUDE'] = lon_list

In [28]:
# find subway station within 5 km 
pa19_df = pa19_df[pa19_df['LATITUDE'].notna()]
pa19_df = pa19_df[pa19_df['LONGITUDE'].notna()]
for i in range (len(df)):
    ss_dist = []
    coords_ss = (df.iloc[i,5], df.iloc[i,6])
    for j in range (len(pa19_df)): 
        coords_pa19 = (pa19_df.iloc[j,17], pa19_df.iloc[j,18]) 
        if distance.distance(coords_ss, coords_pa19).km <=5:
            ss_dist.append(1)
            
        else: 
            ss_dist.append(0)
    pa19_df[df.iloc[i,0]] = ss_dist

pa19_df.head()

,ARTWORKNAM,Nbhd#,Ward#(25),Ward Name (25),ADDRESS,POSTAL_COD,POSTCODE2,X_COORDI,Y_COORDI,PARK_NAME,...,Union Station,Vaughan Metropolitan Centre Station,Victoria Park Station,Warden Station,Wellesley Station,Wilson Station,Woodbine Station,York Mills Station,York University Station,Yorkdale Station
1,HOWARD MONUMENT,87,4,Parkdale-High Park,11 COLBORNE LODGE DR Toronto Canada,M6R 2Z3,M6R2Z3,307899.681,4833197.499,HIGH PARK,...,0,0,0,0,0,0,0,0,0,0
2,FORT ROUILLE MONUMENT,82,10,Spadina-Fort York,1 YUKON PL Toronto Canada,M6K 3C3,M6K3C3,310956.892,4832080.413,NaN,...,1,0,0,0,0,0,0,0,0,0
3,MURAL,76,13,Toronto Centre,60 QUEEN ST W Toronto Canada,M5H 2M3,M5H2M3,314322.726,4834469.244,NaN,...,1,0,0,0,1,0,0,0,0,0
4,ROBERT BURNS MEMORIAL,73,13,Toronto Centre,19 HORTICULTURAL AVE Toronto Canada,M5A 2P2,M5A2P2,313491.916,4835448.235,NaN,...,1,0,0,0,1,0,0,0,0,0
5,WAR OF 1812 MEMORIAL,77,10,Spadina-Fort York,10 NIAGARA ST Toronto Canada,M5V 1C2,M5V1C2,312838.568,4833427.823,VICTORIA MEMORIAL PARK,...,1,0,0,0,1,0,0,0,0,0


## 5. Public Art 

In [39]:
pa_df = pd.read_csv('Public_Art_Cleaned.csv')
pa_df = pa_df[pa_df['Full_Address'].notna()]
geolocator = Nominatim(user_agent = "geoapiExercises")
#find lat and long
lat_list = []
lon_list = []
for i in range(len(pa_df)): 
    place = pa_df.iloc[i,-1]
    location = geolocator.geocode(place)
    if location:
        lat=location.latitude
        lon=location.longitude
    else :
        lat = np.nan
        lon = np.nan
    lat_list.append(lat)
    lon_list.append(lon)
    
pa_df['LATITUDE'] = lat_list
pa_df['LONGITUDE'] = lon_list

In [40]:
pa_df

,_id,FID,Source,Source_ID,Location,Artist,Title,Medium,ArtForm,Status,...,ImageOrientation,ExternalLink,ExternalLinkText,FID2,geometry,FSR,LDU,Full_Address,LATITUDE,LONGITUDE
0,789,107,Percent for Public Art - Private Developer Pro...,7034,"25 York Street, Tower TELUS House Toronto","Rokeby, David / Awad, Michael",Pixel Cube,LED Lights,NaN,Existing,...,landscape,NaN,NaN,1,"{u'type': u'Point', u'coordinates': (-79.38117...",M5J,"2V5,","Union Health, 25, York Street, Harbourfront, S...",43.643372,-79.381245
1,790,108,Percent for Public Art - Private Developer Pro...,5100,"628 Fleet Street, West Harbour City Condos","Krger, Nestor",Pond,Granite and Marble,NaN,Existing,...,landscape,NaN,NaN,2,"{u'type': u'Point', u'coordinates': (-79.40265...",M5V,"1A4,","628, Fleet Street, Fort York, Spadina—Fort Yor...",43.636110,-79.403329
2,791,109,Percent for Public Art - Private Developer Pro...,13005,"875 Morningside Avenue, Toronto Pan Am Sports ...","BGL (Jasmin Bilodeau, Sebastien Giguere, Nicol...",Pool Lane/La velocite de l'eau,NaN,NaN,Existing,...,landscape,NaN,NaN,3,"{u'type': u'Point', u'coordinates': (-79.19427...",M1C,"0C7,","Toronto Pan Am Sports Centre (TPASC), 875, Mor...",43.790793,-79.193699
3,792,110,Percent for Public Art - Private Developer Pro...,98069,Pedestrian Bridge connecting Front Street West...,"Gazitua, Francisco",Puente de Luz / The Bridge,NaN,NaN,Existing,...,landscape,NaN,NaN,4,"{u'type': u'Point', u'coordinates': (-79.39786...",M5N,"3Z3,","CityPlace, Spadina—Fort York, Old Toronto, Tor...",43.639248,-79.396387
4,793,111,Percent for Public Art - Private Developer Pro...,2057,"285 Mutual Street, Radio City Condos","Brener, Roland",Radioville,"Stainless Steel, Light",NaN,Existing,...,landscape,NaN,NaN,5,"{u'type': u'Point', u'coordinates': (-79.37776...",M5B,"1M3,","285, Mutual Street, Church-Wellesley Village, ...",43.663483,-79.378365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,1186,527,Percent for Public Art - Private Developer Pro...,229,"131 McMahon Drive, Ethennonnhawahstihnen Park,...","Lum, Ken",Cracked Ice,Glass and Steel,NaN,Existing,...,landscape,NaN,NaN,398,"{u'type': u'Point', u'coordinates': (-79.37117...",M2K,"1C2,","131, McMahon Drive, Concord Park Place, Don Va...",43.767744,-79.371175
251,1187,528,Percent for Public Art - Private Developer Pro...,230,"131 McMahon Drive, Ethennonnhawahstihnen Park,...","Belmore, Michael",Ebb and Flow,Glass and Steel,NaN,Existing,...,portrait,https://vimeo.com/455876021,External video,399,"{u'type': u'Point', u'coordinates': (-79.37117...",M2K,"1C2,","131, McMahon Drive, Concord Park Place, Don Va...",43.767744,-79.371175
252,1188,529,Percent for Public Art - Private Developer Pro...,231,88 Davenport Road,"Poussin, Pierre",Exported Floras,"Laser-cut Steel and LEDs, 24.5m W x 4.2m H x 0...",NaN,Existing,...,landscape,NaN,NaN,400,"{u'type': u'Point', u'coordinates': (-79.38995...",M5R,"1H4,","The Florian Residences of Upper Yorkville, 88,...",43.673319,-79.390447
253,1189,530,Percent for Public Art - Private Developer Pro...,232,70 Queens Wharf Road,"McClelland, Niall",Zig Zag,"Exterior latex paint, flat bar, + aluminum 3-D...",NaN,Existing,...,landscape,https://llain-etc.com/,External website,401,"{u'type': u'Point', u'coordinates': (-79.40007...",M5V,"2V6,","Queens Wharf Road, Bathurst Quay, Spadina—Fort...",43.638642,-79.399078
